In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import Optional, Literal # Optional and Literal are from the 'typing' module
from pydantic import BaseModel, Field # BaseModel and Field are from the 'pydantic' module

# Load environment variables from a .env file. 
# This securely loads API keys (like OPENAI_API_KEY) from an environment file,
# keeping sensitive information out of the main codebase.
load_dotenv()

True

In [2]:
# Initialize the ChatOpenAI language model. 
# This creates an instance of the OpenAI chat model (e.g., gpt-3.5-turbo or gpt-4,
# depending on your environment configuration and OpenAI setup). This model will
# be used to process text and extract structured information.
model = ChatOpenAI()

In [3]:
# Define the schema for the desired structured output using Pydantic's BaseModel. 
# This class acts as a blueprint for the JSON object we want the LLM to return.
# Pydantic models offer robust data validation, parsing, and serialization.
class Review(BaseModel):
    # 'key_themes': A list of strings. 'Field' provides a description for the LLM
    # to understand what to extract for this field.
    key_themes: list[str] = Field(description="Write down all the key themes discussed in the review in a list")
    # 'summary': A string containing a brief summary.
    summary: str = Field(description="A brief summary of the review")
    # 'sentiment': A string that must be either "pos" (positive) or "neg" (negative).
    # 'Literal' ensures the value is one of these exact strings.
    # The 'Field' description guides the LLM on how to determine sentiment.
    sentiment: Literal["pos", "neg"] = Field(description="Return sentiment of the review either negative or positive")
    # 'pros': An optional list of strings. 'Optional' means it can be a list[str] or None.
    # 'default=None' explicitly sets its default value if not provided by the LLM.
    pros: Optional[list[str]] = Field(default=None, description="Write down all the pros inside a list")
    # 'cons': An optional list of strings, similar to 'pros'.
    cons: Optional[list[str]] = Field(default=None, description="Write down all the cons inside a list")
    # 'name': An optional string for the reviewer's name.
    name: Optional[str] = Field(default=None, description="Write the name of the reviewer")

In [4]:
# Create a structured output model from the base ChatOpenAI model. 
# The 'with_structured_output' method transforms the LLM into one that is instructed
# to return responses conforming to the provided 'Review' Pydantic BaseModel schema.
# LangChain internally converts the Pydantic model into a JSON Schema, which
# advanced LLMs (like newer OpenAI models) can use as a "function call" specification
# to generate a JSON object matching the schema.
structured_model = model.with_structured_output(Review)

d:\Projects\langchain-cookbook\venv\Lib\site-packages\langchain_openai\chat_models\base.py:1857: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [5]:
# Invoke the structured model with the review text.
# The model will process this text and attempt to extract information
# into the fields defined in the 'Review' schema, returning a Pydantic object.
result = structured_model.invoke("""I recently got the Solanki True Wireless Earbuds, and they've been a mixed bag. The sound quality is genuinely impressive, offering crisp highs and decent bass for their size, and they fit surprisingly comfortably in my ears for long listening sessions. The battery life is also solid, easily getting me through a workday.
However, the touch controls are incredibly finicky, often registering accidental taps or failing to respond. The charging case is also a bit bulkier than I'd like, making it less pocket-friendly.

Pros:

Excellent sound quality
Comfortable fit
Good battery life

Review by Ujjwal Solanki
""")

In [6]:
# Print the entire Pydantic model instance. 
# The 'result' object is an instance of the 'Review' Pydantic model,
# populated with the extracted data. When printed, it shows the field names
# and their extracted values in a readable format.
print(result)

key_themes=['sound quality', 'comfort', 'battery life', 'touch controls', 'charging case'] summary='The Solanki True Wireless Earbuds offer impressive sound quality, a comfortable fit, and good battery life. However, the touch controls are finicky and the charging case is bulkier than ideal.' sentiment='pos' pros=['Excellent sound quality', 'Comfortable fit', 'Good battery life'] cons=['Finicky touch controls', 'Bulky charging case'] name='Ujjwal Solanki'
